In [2]:
# https://sagemaker-examples.readthedocs.io/en/latest/step-functions-data-science-sdk/step_functions_mlworkflow_processing/step_functions_mlworkflow_scikit_learn_data_processing_and_model_evaluation.html
# https://sagemaker-examples.readthedocs.io/en/latest/step-functions-data-science-sdk/machine_learning_workflow_abalone/machine_learning_workflow_abalone.html

# https://github.com/aws-samples/data-science-on-aws/blob/main/04a_Create_End_to_End_MLOps_Pipeline.ipynb

In [4]:
# ! pip install -U sagemaker

DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
# pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install stepfunctions  omegaconf  nb-black python-dotenv

  Using cached stepfunctions-2.3.0-py2.py3-none-any.whl
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached nb_black-1.0.7-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 885.1/885.1 kB 8.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
  Obtaining dependency information for black>='19.3' from https://files.pythonhosted.org/packages/21/24/b6ee7df9690e5d6eb6c6bad1e36aa030002c14c921324fad265e89799273/black-23.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 723.4 kB/s eta 0:00:00a 0:00:01
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
  Obtaining dependency information for pathspec>=0.9.0 from https://files.pythonhosted.org/packages/b4/2a/9b1be29146139ef459188f5e420a66e835dda921208db600b7037093891f/pathspec-0.11.2-py3-none-any.whl

In [7]:
!pip uninstall -y PyYAML

Found existing installation: PyYAML 6.0.1
Uninstalling PyYAML-6.0.1:
  Successfully uninstalled PyYAML-6.0.1


In [8]:
pip install PyYAML

  Obtaining dependency information for PyYAML from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 7.7 MB/s eta 0:00:00:00:01
DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
%load_ext lab_black

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import general libreries

In [ ]:
import stepfunctions
from config import config


In [ ]:
REGION = config.region

# Custom Retry Configuration

In [8]:
# This code configures a Boto3 client for Amazon SageMaker with specific timeouts and retry settings, then prints the configured retry settings.
import boto3
from botocore.config import Config

sm_boto = boto3.client(
    "sagemaker",
    config=Config(connect_timeout=5, read_timeout=60, retries={"max_attempts": 60}),
)
print(sm_boto.meta.config.retries)

{'total_max_attempts': 61, 'mode': 'legacy'}


# Sagemaker Configuration

In [10]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [11]:
# SageMaker Session

sagemaker_session = sagemaker.Session(sagemaker_client=sm_boto)
region = sagemaker_session.boto_session.region_name
print(sagemaker_session.sagemaker_client.meta.config.retries)
# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
role = get_execution_role()
bucket = config.s3.bucket
prefix = config.s3.prefix
bucket_path = config.s3.bucket_path
s3_bucket_base_uri = config.s3.s3_bucket_base_uri

bucket_path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
{'total_max_attempts': 61, 'mode': 'legacy'}
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://ck-qa-pldatascience/aws_mlOps/sagemaker-pipeline/stepfunctions/'

# Processing Steps

In [12]:
import os
from sagemaker.sklearn.processing import SKLearnProcessor

import time
import stepfunctions
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.steps import (
    Chain,
    ChoiceRule,
    ModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep,
    Parallel,
)

from sagemaker.processing import ProcessingInput, ProcessingOutput

## Functions

In [13]:
def job_name(jobname):
    """
    Generate a unique job name for an Amazon SageMaker job based on a given 'jobname' and current timestamp.

    Args:
        jobname (str): A descriptive name for the job.

    Returns:
        str: A unique job name incorporating the 'jobname' and timestamp.
    """
    return f"MultiLabelClassification-{jobname}--{time.strftime('%Y%m%d%H%M%S', time.gmtime())}"


def upload_code(bucket_name, prefix_name, script_location):
    """
    Upload code or script to an Amazon S3 bucket for use in SageMaker.

    Args:
        bucket_name (str): The name of the S3 bucket where the code will be uploaded.
        prefix_name (str): The prefix or directory within the S3 bucket where the code will be stored.
        script_location (str): The local path to the code or script file.

    Returns:
        str: The S3 URI of the uploaded code.
    """
    return sagemaker_session.upload_data(
        script_location,
        bucket=bucket_name,
        key_prefix=f"{prefix_name}/{script_location}",
    )

## Config

In [14]:
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the local .env file, including OPENAI_API_KEY
_ = load_dotenv(find_dotenv())

In [15]:
# Set your execution role
workflow_execution_role = os.getenv("SAGEMAKER_WORKFLOW_ROLE")

In [16]:
execution_input = ExecutionInput(
    schema={
        "PreprocessingJobName": str,
    }
)

In [17]:
failed_state_sagemaker_processing_failure = stepfunctions.steps.states.Fail(
    "ML Workflow failed", cause="SageMakerProcessingJobFailed"
)
catch_state_processing = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_processing_failure,
)

## 1.1 Preprocess Job

In [20]:
f"{prefix}/{PREPROCESSING_SCRIPT_LOCATION}"

'aws_mlOps/sagemaker-pipeline/stepfunctions/preprocess/code'

In [18]:
PREPROCESSING_SCRIPT_LOCATION = "preprocess/code"

input_code_preprocess = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=f"{prefix}/{PREPROCESSING_SCRIPT_LOCATION}",
)

In [21]:
output_preprocess = "{}/{}".format(s3_bucket_base_uri, config.s3.preprocess.train)
output_preprocess

's3://ck-qa-pldatascience/aws_mlOps/sagemaker-pipeline/stepfunctions/train.csv'

In [25]:
f"s3://{bucket}/{prefix}/{PREPROCESSING_SCRIPT_LOCATION}/config"

's3://ck-qa-pldatascience/aws_mlOps/sagemaker-pipeline/stepfunctions/preprocess/code/config'

In [26]:
inputs_preprocess = [
    ProcessingInput(
        source=f"{config.s3.s3_bucket_base_uri}/{config.s3.input}",
        destination="/opt/ml/processing/input/data",
        input_name="input-data",
    ),
    ProcessingInput(
        source=input_code_preprocess,
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
    ProcessingInput(
        source=f"s3://{bucket}/{prefix}/{PREPROCESSING_SCRIPT_LOCATION}/config",
        destination="/opt/ml/processing/input/config",
        input_name="code-config",
    ),
]


outputs_preprocess = [
    ProcessingOutput(
        source="/opt/ml/processing/process",
        destination=output_preprocess,
        output_name="train_data",
    )
]

In [27]:
preprocessing_step_name = f"MultLabelClasification - Preprocessing Step {time.strftime('%Y%m%d%H%M%S', time.gmtime())}"


def sklearn_processor(instance_type="ml.m5.xlarge"):
    return SKLearnProcessor(
        framework_version="1.2-1",
        role=role,
        # instance_type="ml.m5.xlarge",
        instance_type="ml.t3.medium",
        instance_count=1,
        # max_runtime_in_seconds=1200,
    )


# Define the SageMaker processing step for data preprocessing.

processing_step = ProcessingStep(
    preprocessing_step_name,
    processor=sklearn_processor(),
    job_name=f"PreprocessingJobName{time.strftime('%Y%m%d%H%M%S', time.gmtime())}",
    inputs=inputs_preprocess,
    outputs=outputs_preprocess,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/run.py"],
    # container_arguments=['--dependencies', 'requirements.txt']
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Create WorkFlow

In [28]:
workflow_graph = Chain([processing_step])


branching_workflow = Workflow(
    name=job_name("MlOpsWorkflow"),
    definition=workflow_graph,
    role=workflow_execution_role,
)

branching_workflow.create()

'arn:aws:states:us-east-1:889859566884:stateMachine:auto-tagging-MlOpsWorkflow--20230919163656'

In [29]:
# Execute workflow
execution = branching_workflow.execute(
    inputs={
        "PreprocessingJobName": job_name("preprocess"),
        # "EvaluationProcessingJobName": evaluation_job_name,  # Each SageMaker processing job requires a unique name,
    }
)


In [30]:
execution.get_output(wait=True)